### 1. 不使用框架搭建CNN

#### 1.1 引入库

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0,4.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload

%autoreload 2

np.random.seed(1)

#### 1.2 zero padding

In [2]:
def zero_pad(X,pad):
    """
    参数：
        X - 图像数据集，维度为（样本数，图像高度，图像宽度，通道数）
        pad - padding的单元数
    返回：
        X_paded - padding后的图像数据集 
    """
    X_paded = np.pad(X,((0,0),(pad,pad),(pad,pad),(0,0)),'constant',constant_values=(0,0))
    return X_paded

In [3]:
# 测试zero_pad
np.random.seed(1)
x = np.random.randn(4,3,3,2)
x_paded = zero_pad(x,2)

print("x.shape = "+str(x.shape))
print("x_paded.shape = "+str(x_paded.shape))

x.shape = (4, 3, 3, 2)
x_paded.shape = (4, 7, 7, 2)


#### 1.3 单步卷积

In [6]:
def conv_single_step(a_slice_prev,W,b):
    """
    参数：
        W - 卷积核
        a_slice_prev - 与卷积核size相同的图像的一部分
        b - bias
    返回：
        Z - 进行一次单步卷积的结果
    """
    temp = np.multiply(a_slice_prev,W)+b
    Z = np.sum(temp)
    
    return Z

In [7]:
# 测试conv_single_step
np.random.seed(1)

a_slice_prev = np.random.randn(4,4,3)
W = np.random.randn(4,4,3)
b = np.random.randn(1,1,1)

Z = conv_single_step(a_slice_prev,W,b)
print("Z = "+str(Z))

Z = -23.16021220252078


#### 1.4 卷积层前向传播

In [12]:
def conv_forward(A_prev,W,b,hparameters):
    """
    参数：
        A_prev - (m,n_h_prev,n_w_prev,n_c_prev)维度
        W - (f,f,n_c_prev,n_c)维度
        b - bias (1,1,1)
        hparameters - 包含stride 和 pad参数的字典
    返回：
        Z - 卷积+bias后的输出
        cache - 中间值 bp时会用到
    """
    # 获取输入的size
    (m,n_h_prev,n_w_prev,n_c_prev) = A_prev.shape
    # 获取卷积核size
    (f,f,n_c_prev,n_c) = W.shape
    
    # 获取stride和pad超参数
    s = hparameters["stride"]
    p = hparameters["pad"]
    
    # 初始化输出Z
    n_h = int((n_h_prev+2*p-f)/s)+1
    n_w = int((n_w_prev+2*p-f)/s)+1
    Z = np.zeros((m,n_h,n_w,n_c))
    
    # padding
    A_prev_padding = zero_pad(A_prev,p)
    
    # 计算Z
    for i in range(m):
        # 取出第i个图像
        a_prev_padding = A_prev_padding[i]
        for h in range(n_h):
            for w in range(n_w):
                for c in range(n_c):
                    # 获得a_slice_prev
                    vertical_start = h*s
                    vertical_end = vertical_start + f
                    horizontal_start = w*s
                    horizontal_end = horizontal_start + f
                    a_slice_prev = a_prev_padding[vertical_start:vertical_end,horizontal_start:horizontal_end,:]
                    
                    # single conv
                    Z[i,h,w,c] = conv_single_step(a_slice_prev,W[:,:,:,c],b[:,:,:,c])
    cache = (A_prev,W,b,hparameters)
    return Z,cache

In [13]:
# 测试conv_forward
np.random.seed(1)
A_prev = np.random.randn(10,4,4,3)
W = np.random.randn(2,2,3,8)
b = np.random.randn(1,1,1,8)

hparameters = {"pad":2,"stride":1}
Z,cache_conv = conv_forward(A_prev,W,b,hparameters)

print("np.mean(Z) = "+str(np.mean(Z)))
print("cache_conv[0][1][2][3] = "+str(cache_conv[0][1][2][3]))

np.mean(Z) = 0.15585932488906465
cache_conv[0][1][2][3] = [-0.20075807  0.18656139  0.41005165]
